In [10]:
import os
import requests
from bs4 import BeautifulSoup
import re
import csv

def save_article_text(url):
    # Set headers to mimic a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Send a GET request to the URL with headers
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the title, subtitle, description, and main text
    title_element = soup.find('title')
    title = title_element.text.strip() if title_element else ""

    subtitle_element = soup.find('meta', attrs={'name': 'description'})
    subtitle = subtitle_element['content'].strip() if subtitle_element and 'content' in subtitle_element.attrs else ""

    description_element = soup.find('meta', attrs={'name': 'og:description'})
    description = description_element['content'].strip() if description_element and 'content' in description_element.attrs else ""

    # Find and exclude unwanted elements by class names or content patterns
    unwanted_elements = soup.find_all(['script', 'style', 'a', 'div', 'span'], class_=['follow-us', 'newsletter', 'advertisement'])
    patterns_to_exclude = ['next article', 'read next', 'correlated']
    for element in unwanted_elements:
        if any(pattern in str(element).lower() for pattern in patterns_to_exclude):
            element.extract()

    # Find and exclude footer container and "All rights reserved" text
    footer_elements = soup.find_all(['footer', 'div'], class_=['footer', 'bottom-footer'])
    for element in footer_elements:
        element.extract()
    all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
    for element in all_rights_reserved_elements:
        element.extract()

    # Find the main text element(s) based on the HTML structure of the page
    main_text_elements = soup.find_all('p')
    main_text = "\n\n".join([element.text.strip() for element in main_text_elements if element.text.strip()])

    # Set the subtitle to the description if it is empty
    if not subtitle:
        subtitle = description.strip()

    # Concatenate the extracted strings
    article_text = f"{title}\n\n{subtitle}\n\n{main_text}"

    return article_text


In [11]:
def create_csv_file(urls):
    # Create a CSV file named "articles.csv"
    filename = "articles.csv"

    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)

        # Iterate over the URLs
        for i, url in enumerate(urls, start=1):
            try:
                # Call save_article_text to get the article text
                article_text = save_article_text(url)

                # Write the article text to the CSV file
                writer.writerow([article_text])

                # Start a new row after every 10 URLs
                if i % 10 == 0:
                    writer.writerow([])  # Add an empty row

                print(f"Article processed: {url}")
            except (requests.exceptions.RequestException, requests.exceptions.HTTPError) as e:
                # Handle the exception and continue with the next URL
                print(f"Error processing URL: {url}")
                print(f"Error message: {str(e)}")
                continue

    print(f"CSV file created: {filename}")

In [12]:
# List of URLs
urls = [
"https://indianexpress.com/article/political-pulse/karnataka-bjp-manifesto-committee-chief-nrc-k-sudhakar-8586008/",
"https://indianexpress.com/article/opinion/editorials/express-view-on-bjps-karnataka-manifesto-high-on-hindutva-8586138/",
"https://sports.ndtv.com/ipl-2023/csk-youngsters-boundary-rope-catch-divides-fans-watch-out-for-ms-dhonis-reaction-video-3994211",
"https://timesofindia.indiatimes.com/sports/cricket/ipl/top-stories/ipl-2023-why-ms-dhoni-isnt-batting-up-the-order-for-chennai-super-kings/articleshow/99923656.cms?from=mdr",
"https://www.insidesport.in/lsg-vs-csk-live-score-today-ipl-match-lucknow-super-giants-vs-chennai-super-kings-at-330-pm-follow-ipl-2023-live-score/",
"https://www.cricbuzz.com/cricket-news/126429/the-ipl-pulse-cricbuzzcom",
"https://www.timesnownews.com/india/row-over-the-kerala-story-muslim-body-announces-rs-1-cr-reward-sets-one-condition-article-99911669",
"https://www.news18.com/movies/vipul-shah-calls-the-kerala-story-an-uncomfortable-truth-says-if-anybody-has-reason-to-ban-it-despite-cbfc-clearance-exclusive-7689931.html",
"https://www.hindustantimes.com/india-news/the-kerala-story-islamic-state-syria-32000-women-vipul-shah-shashi-tharoor-pinarayi-vijayan-101682990800269.html",
"https://www.indiatoday.in/movies/regional-cinema/story/kerala-government-ban-on-controversial-the-kerala-story-is-isis-exhibitors-say-not-a-good-precedent-2366920-2023-05-01",
"https://indianexpress.com/article/political-pulse/with-karnataka-four-states-counting-how-bjp-is-advancing-its-ucc-agenda-8585940/",
"https://www.ndtv.com/india-news/karnataka-assembly-election-2023-congress-promises-action-including-ban-on-organisations-spreading-hatred-in-manifesto-cites-bajrang-dal-pfi-3996940",
"https://indianexpress.com/article/opinion/editorials/express-view-on-bjps-karnataka-manifesto-high-on-hindutva-8586138/",
"https://www.onmanorama.com/news/kerala/2023/05/02/athira-suicide-cyberbullying-marriage-proposals-brother-statement.html",
"https://www.deccanherald.com/national/north-and-central/ed-mentions-aap-mp-raghav-chadha-in-liquor-policy-case-chargesheet-1214907.html",
"https://timesofindia.indiatimes.com/entertainment/hindi/bollywood/news/shraddha-kapoor-wins-hearts-with-her-no-makeup-look-netizens-admire-her-simplicity-and-unfiltered-personality/articleshow/99918268.cms?from=mdr",
"https://www.koimoi.com/bollywood-news/when-salman-khan-fired-a-shot-at-shah-rukh-khan-said-koi-nahin-uthega-sabko-maar-doonga-after-someone-said-we-will-all-be-in-a-soup-lets-rush-out/",
"https://timesofindia.indiatimes.com/entertainment/hindi/bollywood/news/salman-khan-addresses-rumours-of-romantic-relationship-with-shehnaaz-gill-i-told-her-to-move-on-par-iska-ye-matlab-nahi-ke-bring-it-on/articleshow/99917987.cms?from=mdr",
"https://www.ndtv.com/entertainment/salman-khan-on-becoming-a-father-i-was-planning-but-3994992",
"https://indianexpress.com/article/india/kerala-woman-ends-life-cyberbullying-male-friend-booked-8587307/",
"https://www.sportskeeda.com/football/news-no-championship-99-scored-robert-lewandowski-s-ex-teammate-brutally-mocks-cristiano-ronaldo-al-nassr-win-al-raed",
"https://www.goal.com/en-in/news/cristiano-ronald-brutally-mocked-social-media-former-al-nassr-midfielder-goal-celebration-al-raed/blt975ab5c817c49966",
"https://indianexpress.com/article/sports/football/adrian-mierzejewski-mocked-ronaldo-on-social-media-after-al-nassar-wins-against-al-raed-8586965/",
"https://dailypost.ng/2023/05/01/mierzejewski-mocks-cristiano-ronaldo-after-al-nassr-win/",
"https://thewire.in/politics/karnataka-elections-congress-bjp",
"https://timesofindia.indiatimes.com/city/mysuru/karnataka-assembly-elections-high-decibel-blitzkrieg-seeks-to-derail-congress/articleshow/99919046.cms",
"https://www.thehindu.com/elections/karnataka-assembly/election-agenda-it-is-turning-out-to-be-bjps-national-vs-congress-local-in-the-rural-hinterland/article66799609.ece",
"https://theprint.in/elections/karnataka-assembly-elections/in-karnataka-its-a-battle-of-congresss-perception-management-versus-bjps-micro-management/1546586/",
"https://indianexpress.com/article/cities/delhi/former-delhi-deputy-cm-sisodia-got-ex-cjis-legal-opinion-on-excise-policy-removed-ed-8586295/",
"https://www.timesnownews.com/delhi/delhi-excise-policy-case-over-rs-192-crore-crime-proceeds-generated-ed-explains-indo-spirits-connection-article-99921527",
"https://www.thehindu.com/elections/karnataka-assembly/karnataka-assembly-elections-2023-congress-releases-manifesto-promises-to-repeal-anti-people-laws-passed-by-bjp/article66802659.ece",
"https://indianexpress.com/article/opinion/editorials/express-view-on-bjps-karnataka-manifesto-high-on-hindutva-8586138/",
"https://timesofindia.indiatimes.com/entertainment/hindi/bollywood/news/met-gala-2023-alia-bhatt-looks-mesmerising-in-her-first-look/articleshow/99918353.cms?from=mdr",
"https://www.ndtv.com/entertainment/met-gala-2023-alia-bhatt-floats-like-an-angel-in-debut-did-someone-say-slay-3996663",
"https://indianexpress.com/article/lifestyle/fashion/priyanka-chopra-in-black-and-alia-bhatt-in-white-south-asia-talking-point-met-gala-8586658/",
"https://zeenews.india.com/cricket/virat-kohlis-romance-revenge-blows-flying-kiss-to-anushka-sharma-and-mimics-gautam-gambhirs-crowd-silencing-gesture-in-ipl-2023-rcb-vs-lsg-2601560.html",
"https://www.insidesport.in/cricket-lsg-vs-rcb-ipl-2023-virat-kohli-anushka-sharma-get-cosy-at-rcb-bar-ahead-of-crunch-lucknow-super-giants-clash-see-pics/",
"https://indianexpress.com/article/cities/chandigarh/jalandhar-aap-promised-monthly-allowance-8586688/",
"https://timesofindia.indiatimes.com/entertainment/hindi/bollywood/news/naga-chaitanya-talks-about-his-biggest-regret-steers-clear-of-his-divorce-with-samantha-ruth-prabhu/articleshow/99917121.cms?from=mdr",
"https://www.news18.com/movies/naga-chaitanya-reveals-biggest-regret-in-life-after-divorce-from-samantha-ruth-prabhu-7685767.html",
"https://theprint.in/opinion/politically-correct/brij-bhushan-sharan-singh-is-no-saint-but-bjp-has-reasons-to-look-away-from-wrestlers-protest/1548795/",
"https://www.deccanherald.com/opinion/union-government-s-indifference-towards-protesting-wrestlers-is-baffling-1214923.html",
"https://indianexpress.com/article/entertainment/tamil/aishwarya-lekshmi-there-is-some-level-of-unfairness-towards-the-indigenous-empires-of-india-8585596/",
"https://www.koimoi.com/box-office/ponniyin-selvan-2-box-office-day-4-enjoys-a-solid-monday-with-20-crore-on-board-enters-the-100-crore-club/",
"https://indianexpress.com/article/entertainment/tamil/ponniyin-selvan-2-box-office-collection-day-4-mani-ratnam-film-crosses-rs-200-cr-globally-beats-vijay-varisu-beast-8586500/",
"https://www.news18.com/entertainment/tamil-cinema/ponniyin-selvan-2-man-flies-all-the-way-from-japan-to-watch-karthi-7700701.html"
"https://www.livemint.com/news/india/ipl-fans-engage-in-brawl-during-dc-vs-srh-match-in-delhi-watch-11682931471319.html",
"https://m.cricbuzz.com/cricket-commentary/66526/srh-vs-dc-40th-match-indian-premier-league-2023",
"https://www.news18.com/cricketnext/gt-vs-dc-probable-xis-ipl-2023-head-to-head-record-gujarat-titans-vs-delhi-capitals-7692493.html",
"https://www.hindustantimes.com/cricket/priyam-garg-to-continue-in-side-prithvi-shaw-to-be-benched-again-dc-s-likely-xi-vs-gt-in-ipl-2023-101682939276305.html",
"https://www.deccanherald.com/national-politics/congress-rahul-gandhi-bjp-narendra-modi-mallikarjun-kharge-priyank-kharge-sharad-pawar-karnataka-assembly-elections-may-3-1215143.html",
"https://www.ndtv.com/india-news/karnataka-election-pm-narendra-modi-shreds-congress-says-its-history-is-about-appeasing-terrorists-3997374",
"https://www.thehindu.com/news/national/karnataka/top-news-developments-in-karnataka-on-may-3-2023/article66806593.ece",
"https://www.deccanherald.com/election/karnataka/shahi-parivar-wants-to-make-karnataka-no1-atm-pm-modi-tears-into-congress-1215194.html",
"https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/gujarat-titans-vs-delhi-capitals-44th-match-1359518/match-report",
"https://www.rediff.com/cricket/report/photos-gujarat-titans-vs-delhi-capitals-hardik-aman-shami-khan-gill-warner/20230502.htm",
"https://m.cricbuzz.com/cricket-news/126449/i-tried-by-best-but-could-not-capitalise-hardik-pandya",
"https://www.thequint.com/podcast/t20-podcast-ayaz-memon-gujarat-delhi-cricket",
"https://www.indiatoday.in/india/story/the-kerala-story-release-controversy-islamic-states-conversion-muslim-women-facts-pinarayi-vijayan-congress-2367245-2023-05-02",
"https://www.greaterkashmir.com/national-2/the-kerala-story-after-row-32000-missing-women-changed-to-3",
"https://www.indiatoday.in/elections/karnataka-assembly-polls-2023/story/bajrang-dal-ban-congress-manifesto-activists-house-poster-dogs-karnataka-assembly-elections-2367731-2023-05-02",
"https://www.ndtv.com/india-news/karnataka-assembly-election-2023-pm-narendra-modi-says-unfortunate-that-congress-has-decided-to-lock-bajrangbali-after-partys-karnataka-manifesto-cite-3997633",
"https://www.freepressjournal.in/bhopal/bhopal-bajrang-dal-threatens-stir-if-banned",
"https://www.ndtv.com/india-news/anurag-thakur-tried-to-suppress-matter-wrestler-vinesh-phogat-amid-protest-3999914",
"https://www.hindustantimes.com/india-news/mahavir-phogat-wrestlers-protest-aamir-khan-geeta-phogat-jantar-mantar-101683081595769.html",
"https://indianexpress.com/article/entertainment/bollywood/salman-khan-his-blessings-essential-for-survival-film-industry-8587139/",
"https://www.koimoi.com/bollywood-news/raj-kapoors-awara-earned-35-million-at-the-worldwide-box-office-in-1968-being-highest-grossing-non-english-film-since-1940-shekhar-kapur-shares-a-trivia-to-be-proud-of/",
"https://timesofindia.indiatimes.com/entertainment/hindi/bollywood/news/salman-khan-takes-a-jibe-at-arbaaz-khan-and-sohail-khans-failed-marriages-heres-what-he-said/articleshow/99952154.cms?from=mdr",
"https://www.ndtv.com/entertainment/salman-khan-on-dating-rumours-with-shehnaaz-gill-told-her-to-move-on-that-doesn-t-mean-bring-it-on-3997580",
"https://indianexpress.com/article/entertainment/bollywood/salman-khan-jokes-about-brothers-arbaaz-and-sohails-divorces-tells-kapil-sharma-they-dont-listen-to-me-8586755/",
"https://theprint.in/opinion/rahul-gandhi-to-metoo-british-era-criminal-defamation-law-is-only-about-power-of-the-elite/1549855/",
"https://www.hindustantimes.com/india-news/swot-analysis-can-the-congress-capture-power-in-karnataka-elections-101683051677896.html",
"https://indianexpress.com/article/opinion/columns/karnataka-elections-2023-what-the-political-debate-ignores-8588226/",
"https://www.espn.in/football/paris-saint-germain-frapsg/story/4938998/psg-consider-fining-lionel-messi-over-missing-training-for-unauthorised-saudi-trip-sources",
"https://www.goal.com/en-in/news/lionel-messi-two-week-suspension-psg-trip-saudi-arabia/blt21227b2ac3ad9043",
"https://www.indiatoday.in/sports/football/story/messi-suspended-for-two-weeks-after-trip-to-saudi-arabia-2367777-2023-05-03",
"https://www.sportskeeda.com/football/news-that-guy-fraud-bad-first-minute-chelsea-fans-rip-star-dreadful-performance-3-1-loss-arsenal",
"https://www.goal.com/en-in/lists/what-chance-aubameyang-have-winners-losers-lampard-former-arsenal-captain-disastrous-chelsea-showing/blt691676b7f4fac275",
"https://www.barcablaugranes.com/2023/5/2/23708564/barcelona-osasuna-la-liga-winners-and-losers",
"https://www.goal.com/en-in/lists/barcelona-player-ratings-osasuna-jordi-alba-robert-lewandowski/bltd8bef0ca6f6cb986",
"https://www.ndtv.com/karnataka-news/karnataka-assembly-election-2023-ndtv-lokniti-csds-survey-how-siddaramaiah-basavaraj-bommai-rank-as-chief-minister-choices-3999104",
"https://www.hindustantimes.com/india-news/swot-analysis-can-the-congress-capture-power-in-karnataka-elections-101683051677896.html",
"https://timesofindia.indiatimes.com/entertainment/hindi/bollywood/news/alia-bhatt-gets-mistaken-for-aishwarya-rai-bachchan-by-paps-at-met-gala-2023-she-handles-the-situation-with-grace/articleshow/99944118.cms?from=mdr",
"https://www.news18.com/movies/alia-bhatt-faces-flak-over-met-gala-debut-after-prabal-gurung-says-it-was-my-friend-karan-johar-7706029.html",
"https://www.hindustantimes.com/entertainment/bollywood/alia-bhatt-gets-mistaken-for-aishwarya-rai-by-paparazzi-at-met-gala-2023-101683042346921.html",
"https://www.ndtv.com/entertainment/met-gala-2023-alia-bhatts-viral-reaction-to-fan-chanting-i-love-you-in-new-york-3997466",
"https://indianexpress.com/article/cities/chandigarh/haryana-bhavya-bishnoi-engaged-to-ias-officer-pari-bishnoi-8587931/",
"https://www.123telugu.com/mnews/ntr30-cinematographer-gives-a-major-update.html",
"https://odishatv.in/news/entertainment/ntr30-jr-ntr-wraps-up-major-action-sequences-in-2nd-schedule-of-shooting-203247",
"https://telugucinema.com/news/tv-actress-chaithra-rai-cast-in-ntr30",
"https://www.hindustantimes.com/cricket/tewatia-left-inconsolable-ponting-wild-celebration-ishant-scripted-dcs-unthinkable-last-over-win-vs-gt-101683049710695.html",
"https://www.telegraphindia.com/india/hearing-of-rahul-gandhis-plea-for-stay-on-conviction-in-defamation-case-begins-in-gujarat-high-court/cid/1933833",
"https://www.ndtv.com/india-news/no-interim-relief-for-rahul-gandhi-from-gujarat-high-court-in-defamation-case-3998207",
"https://www.barandbench.com/news/litigation/rahul-gandhi-against-conviction-live-updates-gujarat-high-court",
"https://www.firstpost.com/opinion/the-kerala-story-brutal-reality-of-love-jihad-weaknesses-of-hindu-society-hit-hard-enough-to-trigger-a-rude-awakening-12537992.html",
"https://www.indiatoday.in/india/story/the-kerala-story-release-controversy-islamic-states-conversion-muslim-women-facts-pinarayi-vijayan-congress-2367245-2023-05-02",
"https://indianexpress.com/article/explained/how-accurate-are-the-claims-made-by-the-kerala-story-8586042/",
"https://www.deccanherald.com/opinion/union-government-s-indifference-towards-protesting-wrestlers-is-baffling-1214923.html",
"https://www.ndtv.com/india-news/anurag-thakur-tried-to-suppress-matter-wrestler-vinesh-phogat-amid-protest-3999914",
"https://www.hindustantimes.com/india-news/mahavir-phogat-wrestlers-protest-aamir-khan-geeta-phogat-jantar-mantar-101683081595769.html",
"https://indianexpress.com/article/cities/bangalore/narendra-modi-bengaluru-roadshow-karnataka-polls-2023-8590575/",
"https://www.deccanherald.com/election/karnataka/pm-modi-s-mega-36-km-road-show-in-bengaluru-on-may-6-1215276.html",
"https://www.daijiworld.com/news/newsDisplay?newsID=1076479",
"https://www.cricbuzz.com/cricket-news/126447/ishant-denies-tewatia-as-dc-clinch-low-scoring-thriller-cricbuzzcom",
"https://sports.ndtv.com/ipl-2023/petrol-over-mohammed-shamis-hilarious-response-after-four-wicket-heroics-vs-delhi-capitals-4000161",
"https://www.rediff.com/cricket/report/dcs-aman-makes-most-of-opportunity-ipl-2023/20230503.htm",
"https://www.cricbuzz.com/cricket-news/126449/i-tried-by-best-but-could-not-capitalise-hardik-pandya-cricbuzzcom",
"https://theprint.in/politics/sena-ubt-says-sharad-pawars-criticism-of-uddhav-his-personal-opinion-but-leaders-defend-party-chief/1554068/",
"https://www.ndtv.com/india-news/ncps-key-meeting-tomorrow-to-pick-new-chief-two-days-after-sharad-pawars-shock-resignation-sources-4003635",
"https://indianexpress.com/article/opinion/columns/sharad-pawars-resignation-the-chanakya-has-triumphs-again-8589354/",
"https://www.bollywoodlife.com/news-gossip/deepika-padukone-changes-her-social-media-dp-to-a-picture-of-clouds-netizens-wonder-about-her-move-entertainment-news-2421238/",
"https://www.news18.com/opinion/karnataka-elections-rs-2000-for-women-sop-may-have-seen-it-home-but-cong-just-had-to-do-the-bajrang-dal-own-goal-7716355.html",
"https://www.ndtv.com/karnataka-news/veerappa-moily-on-bajrang-dal-row-congress-leader-says-karnataka-manifesto-has-no-proposal-to-ban-it-4003573",
"https://timesofindia.indiatimes.com/elections/assembly-elections/karnataka/news/pm-modis-chant-posters-power-bajrang-backlash-against-congress-in-karnataka/articleshow/99970794.cms",
"https://www.ndtv.com/entertainment/deepika-padukone-just-changed-her-insta-dp-and-the-internet-has-some-thoughts-4000268",
"https://odishatv.in/news/entertainment/katrina-kaif-s-public-appearances-lessened-after-marriage-to-vicky-kaushal-here-s-the-reason-203325",
"https://www.koimoi.com/bollywood-news/katrina-kaif-staying-away-from-limelight-making-few-public-appearances-has-a-connection-with-salman-khan-led-tiger-3/",
"https://indianexpress.com/article/entertainment/bollywood/parineeti-chopra-and-raghav-chadha-wave-at-fans-as-they-watch-a-ipl-match-in-mohali-see-pics-video-8590467/",
"https://sports.ndtv.com/ipl-2023/raghav-chadha-parineeti-chopra-spotted-watching-ipl-2023-game-in-mohali-twitter-reacts-4002353",
"https://www.rediff.com/cricket/report/ipl-2023-parineeti-and-raghav-chadha-turn-heads-at-ipl-match/20230504.htm",
"https://thewire.in/politics/bajrang-punia-bajrang-dal-narendra-modi-wrestlers",
"https://www.ndtv.com/india-news/wrestlers-allege-night-attack-by-delhi-cops-court-date-today-10-facts-4003307",
"https://www.indiatoday.in/law/story/fir-served-purpose-plea-supreme-court-closes-proceedings-ask-wrestlers-approach-lower-courts-2368480-2023-05-04",
"https://www.news18.com/opinion/opinion-why-congress-finds-itself-on-back-foot-despite-anti-incumbency-hitting-bjp-in-karnataka-7718851.html",
"https://www.hindustantimes.com/india-news/bjp-faces-challenges-in-retaining-power-in-karnataka-assembly-polls-with-focus-on-vokkaliga-dominated-old-mysuru-region-101683140434654.html",
"https://www.ndtv.com/karnataka-news/5-reasons-why-bjp-may-prove-pollsters-wrong-in-karnataka-4001362",
"https://www.newindianexpress.com/opinions/2023/may/03/embargoed-a-ktaka-election-results-copy-in-advance-2571446.html",
"https://www.bavarianfootballworks.com/2023/5/3/23709549/bayern-munich-opinion-paris-saint-germain-leo-messi-hate-saudi-arabia-qatar-fc-barcelona-transfer",
"https://sports.ndtv.com/football/paris-saint-germain-set-for-lionel-messi-divorce-after-suspending-superstar-for-unauthorised-trip-to-saudi-arabia-4002634",
"https://www.goal.com/en-in/news/lionel-messi-no-closer-barcelona-return-psg-exit-huge-saudi-offer-mls-interest-premier-league/blt01df0e4a0b56b194",
"https://www.news18.com/movies/karan-johar-writes-dear-opinion-after-trolls-accused-him-of-sending-alia-bhatt-to-met-gala-7719529.html",
"https://www.rediff.com/movies/report/tara-fatima-watch-the-sound-of-music/20230504.htm",
"https://timesofindia.indiatimes.com/videos/entertainment/hindi/janhvi-kapoor-is-still-using-this-old-photo-of-mom-sridevi-as-her-phones-wallpaper-heres-proof/videoshow/99980516.cms?from=mdr",
"https://timesofindia.indiatimes.com/entertainment/malayalam/movies/news/south-buzz-pune-police-face-backlash-from-ar-rahman-fans-for-disrupting-his-concert-sita-ramam-wins-best-film-jury-award-at-the-13th-dadasaheb-phalke-film-festival-anuragam-trailer-out-now/articleshow/99963211.cms?from=mdr",
"https://www.hindustantimes.com/entertainment/music/ar-rahman-concert-row-cop-who-went-on-stage-and-stopped-show-midway-reveals-why-he-did-it-101683195463876.html",
"https://indianexpress.com/article/entertainment/bollywood/ar-rahman-pune-concert-stopped-policeman-interview-8590609/",
"https://www.india.com/news/india/shimla-municipal-corporation-election-result-2023-live-updates-counting-of-votes-102-candidates-from-34-wards-shimla-civic-polls-result-smc-6029966/",
"https://timesofindia.indiatimes.com/city/shimla/shimla-municipal-corporation-polls-result-congress-wins-eight-seats-bjp-4-counting-under-way-for-remaining-seats/articleshow/99980662.cms",
"https://www.hindustantimes.com/india-news/shimla-municipal-corporation-election-2023-result-live-101683159766673.html",
"https://www.deccanherald.com/national/national-politics/sanjay-singhs-name-in-excise-policy-case-chargesheet-not-mistake-but-deliberate-aap-1215260.html",
"https://indianexpress.com/article/opinion/columns/the-kerala-story-the-film-poses-no-challenge-to-keralas-diverse-social-fabric-8589775/",
"https://www.hindustantimes.com/india-news/tamil-nadu-on-high-alert-as-controversial-film-the-kerala-story-set-for-pan-india-release-on-may-5-101683140729310.html",
"https://www.onmanorama.com/news/kerala/2023/05/04/no-ban-the-kerala-story-government-boycott.html",
"https://www.news18.com/opinion/opinion-the-kerala-story-is-just-a-movie-the-love-jihad-rot-goes-deeper-7719109.html",
"https://www.indiatoday.in/law/story/fir-served-purpose-plea-supreme-court-closes-proceedings-ask-wrestlers-approach-lower-courts-2368480-2023-05-04",
"https://www.ndtv.com/india-news/wrestlers-allege-night-attack-by-delhi-cops-court-date-today-10-facts-4003307",
"https://thewire.in/politics/bajrang-punia-bajrang-dal-narendra-modi-wrestlers",
"https://www.sportskeeda.com/cricket/3-reasons-rahul-tewatia-can-finisher-india-requires-t20is",
"https://sports.ndtv.com/ipl-2023/petrol-over-mohammed-shamis-hilarious-response-after-four-wicket-heroics-vs-delhi-capitals-4000161",
"https://www.cricbuzz.com/cricket-news/126449/i-tried-my-best-but-could-not-capitalise-hardik-pandya"
]

create_csv_file(urls)

/tmp/ipykernel_6577/2665977363.py:41: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))


Article processed: https://indianexpress.com/article/political-pulse/karnataka-bjp-manifesto-committee-chief-nrc-k-sudhakar-8586008/
Article processed: https://indianexpress.com/article/opinion/editorials/express-view-on-bjps-karnataka-manifesto-high-on-hindutva-8586138/
Article processed: https://sports.ndtv.com/ipl-2023/csk-youngsters-boundary-rope-catch-divides-fans-watch-out-for-ms-dhonis-reaction-video-3994211
Article processed: https://timesofindia.indiatimes.com/sports/cricket/ipl/top-stories/ipl-2023-why-ms-dhoni-isnt-batting-up-the-order-for-chennai-super-kings/articleshow/99923656.cms?from=mdr
Article processed: https://www.insidesport.in/lsg-vs-csk-live-score-today-ipl-match-lucknow-super-giants-vs-chennai-super-kings-at-330-pm-follow-ipl-2023-live-score/
Article processed: https://www.cricbuzz.com/cricket-news/126429/the-ipl-pulse-cricbuzzcom
Article processed: https://www.timesnownews.com/india/row-over-the-kerala-story-muslim-body-announces-rs-1-cr-reward-sets-one-conditi